# Password similarity measure using word embeddings with FastText

##### Author: _Karina Chichifoi_

## 0. Introduction

The purpose of this project is to see how similar are the passwords associated to a specific user. Word embeddings are useful for this task, because they can give some additional information about word contexts.

In this notebook FastText is used as word embedding model. It represents each word as an _n-gram_ of characters. This method allows to understand the meaning of short words, and also of prefixes and suffixes, making the process of measuring the similarity of passwords easier.

## 1. Preconditions

Before training FastText, the following operation were performed on a 43 GB breach:
- Removed passwords longer than 30 characters or shorter than 4.
- Removed non ASCII printable characters in a password.
- Removed bot, which are recognisable by the same mail used more than 100 times.
- Removed HEX passwords (identified by `$HEX[]` and `\x`).
- Removed HTML char set.
- Removed mail which appear more than 100 times and less than 2.
- Removed mail with non-valid password.

After that the passwords were converted in a key-press sequence, thanks to ```word2keypress``` python library. In this way is easier to see more similarities between two passwords.
The results were saved in a csv file, which will be used as FastText dataset, in this way:

```sample@gmail.com: ["'97314348'", "'voyager1'"]```



## 2 Environment setup

First of all import all useful libraries.

In [1]:
from gensim.models import FastText
import time
import sys
from pathlib import Path
from PasswordRetriever import PasswordRetriever

```PasswordRetriever``` is a helper class which retrieves a list of passwords from the cleaned csv file.

## 3. Train FastText model

### 3.1 FastText parameters

Choose the parameters for FastText embedding model.

In [2]:
negative = 5
subsampling = 1e-3
min_count = 10
min_n = 1  
max_n = 4  
SIZE = 200  # dimension of the model
sg = 1

In this case it was used the _skip-gram approach_ (`sg = 1`) with _negative sampling_.

- _Skip-gram model_ is chosen, as the distributed representation of the input word is used to predict the context.
_Skip-gram model_ works better with subword information, so it is recommended for learning passwords and rare words in general.

- _Negative sampling_ makes the training faster. Each training sample updates only a small percentage of the model's weights. For larger datasets (like this case) it is recommended to set _negative sampling_ between 2 and 5.

- The dimension of vectors is set to ```200```, in order to have train the model faster. Generally it is recommended to have `SIZE = 300`.  

- The ```subsampling``` ignores the most frequent password (more than 1000 occurrences).

- ```min_count``` represents the minimum number of occurences of a password in the training dataset.

- ```min_n``` and ```max_n``` are the number of respectively _minimum_ and _maximum n-grams_.
    - _N-grams_ are used in statistical natural language processing to predict a word and/or the context of a word. 
    In this case they represent a contiguous sequence of n characters and their purpose is to give subword information.

    For example a password `w = qwerty` with `min_n = 4` and `m_max = 5`, will have the following n-grams.
    `zw = {<qwe, qwer, wert, erty, rty>, <qwer, qwert, werty, erty>}`

    NB `<` and `>` are considered as characters.

### 3.2 Training FastText

The list of password related to every user is obtained from the helper class `PasswordRetriever`.
FastText is ready to train the model, based on `password_list`.

In [3]:
filename='../train.csv'
password_list = PasswordRetriever(filename)
start = time.time()
trained_model = FastText(password_list, size=SIZE, min_count=min_count, workers=12,
                         negative=negative, sample=subsampling, window=20,
                         min_n=min_n, max_n=max_n)
end = time.time()
print("Time taken: {}".format(end - start))

Processed 0 lines in 3.981590270996094e-05 seconds.
Processed 1000000 lines in 8.32761836051941 seconds.
Processed 2000000 lines in 8.211445093154907 seconds.
Processed 3000000 lines in 8.039046287536621 seconds.
Processed 4000000 lines in 8.645042657852173 seconds.
Processed 5000000 lines in 8.688397884368896 seconds.
Processed 6000000 lines in 8.762989521026611 seconds.
Processed 7000000 lines in 9.28832745552063 seconds.
Processed 8000000 lines in 9.370487928390503 seconds.
Processed 9000000 lines in 8.766379356384277 seconds.
Processed 10000000 lines in 8.871519088745117 seconds.
Processed 11000000 lines in 8.899347305297852 seconds.
Processed 12000000 lines in 8.987047910690308 seconds.
Processed 13000000 lines in 9.068806409835815 seconds.
Processed 14000000 lines in 9.055849313735962 seconds.
Processed 15000000 lines in 10.101937294006348 seconds.
Processed 16000000 lines in 9.06157922744751 seconds.
Processed 17000000 lines in 9.10492467880249 seconds.
Processed 18000000 lines 

### 3.3 Saving the model

Finally, the trained model is saved as `.bin`.

In [10]:
from gensim.models.fasttext import save_facebook_model

In [13]:
save_facebook_model(trained_model, "model_password_similarity.bin")
print("Model saved successfully.")

Model saved successfully.


## 4. Compressing the model

The trained model has 4.8GB. There are some problems about the size:
- Too much space occupied in memory.
- It is harder to use the model in client/server architectures. Everyone should use this model, which can be sent as a payload. Embeddings are not reversible, and they guarantee password anonymization.

For these reasons, `compress_fasttext` is used.

In [1]:
import gensim
import compress_fasttext
# from gensim.models.fasttext import load_facebook_vectors

 In order to obtain a compressed model, without impacting significantly on performances, _product quantitation_ and _feature selection_ are applied.
 
 - _Feature selection_ is the process of selecting a subset of relevant features for use in model construction or in other words, the selection of the most important features.

 - _Product quantization_ is a particolar type of _vector quantization_, a lossy compression technique used in speech and image coding. A product quantizer can generate anexponentially large codebook at very low memory/time cost.

In [2]:
big_model = gensim.models.fasttext.load_facebook_vectors('model_password_similarity.bin')
small_model = compress_fasttext.prune_ft_freq(big_model, pq=True)
small_model.save('compressed_model')

The compressed_model is 20MB.

## References
<a name="gensim-fasttext">[1]</a> [Gensim FastText documentation](https://radimrehurek.com/gensim/models/fasttext.html)

<a name="cbow_skipgram">[2]</a> [Differences between cbow and skipgram](https://fasttext.cc/docs/en/unsupervised-tutorial.html)

<a name="negative_sampling">[3]</a> [Negative sampling](http://mccormickml.com/2017/01/11/word2vec-tutorial-part-2-negative-sampling/)

<a name="compress_fasttext">[4]</a> [Compress FastText package](https://pypi.org/project/compress-fasttext/)

<a name="feature-selection">[5]</a> [Feature selection](https://towardsdatascience.com/feature-selection-and-dimensionality-reduction-f488d1a035de)

<a name="product-quantization">[6]</a> [Product quantization](https://www.microsoft.com/en-us/research/wp-content/uploads/2013/11/pami13opq.pdf)

<a name="paper-bijeeta">[7]</a> [PPSM description from the paper "Beyond Credential Stuffing: Password Similarity Models using Neural Networks"](https://www.cs.cornell.edu/~rahul/papers/ppsm.pdf)

<a name="repo-bijeeta">[8]</a> [PPSM repository from the paper "Beyond Credential Stuffing: Password Similarity Models using Neural Networks"](https://github.com/Bijeeta/credtweak)